# Inference tests

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append("../src")

from stochastic_glv_generator import *
from experiments import *
from inference import *

In [2]:
n = 5
seed = 1
r_max = 1.
A_diag_std = 1.
A_off_diag_std = 10.

p = sort_glv_params(n, seed, r_max, A_diag_std, A_off_diag_std)
r = p[:n]
A = p[n:].reshape((n, n))
x_eq = -np.linalg.inv(A)@r
print(x_eq)

np.random.seed(1)
init_perturb = np.random.choice([-1., 1.], n)*np.random.uniform(0.3, 0.6, n)
perturb_scale_list = np.array([1.])
t_samp_list = [np.sort(np.random.uniform(0, 20, 100)) for i in range(3)]
meas_noise_list = np.array([0.01])
repetitions = 1

env_noise = 0.05
t0 = 0
dt = 1e-2

replicates = gen_replicates(p, env_noise, init_perturb, perturb_scale_list, t0, dt, t_samp_list, meas_noise_list, repetitions, seed=0, scale_meas_noise_by_abund=True)
replicates[replicates["dataset"]==0]

new matrix 2[0.15229594 0.2937585  0.04524194 0.08828342 0.078074  ]
3/3                                                                                                 

,dataset,perturbation_scale,measurement_noise,time,sp1,sp2,sp3,sp4,sp5
0,0.0,1.0,0.01,0.180160,0.199071,0.426655,0.028356,0.052023,0.111200
1,0.0,1.0,0.01,0.233384,0.204471,0.412241,0.029233,0.055469,0.110983
2,0.0,1.0,0.01,0.596027,0.197935,0.351575,0.029428,0.058684,0.102503
3,0.0,1.0,0.01,0.683426,0.198191,0.347596,0.030413,0.057781,0.101987
4,0.0,1.0,0.01,0.906915,0.194655,0.332516,0.031326,0.060624,0.098811
...,...,...,...,...,...,...,...,...,...
95,0.0,1.0,0.01,18.710781,0.147125,0.294904,0.040792,0.085408,0.078542
96,0.0,1.0,0.01,18.782556,0.148895,0.293053,0.040775,0.085172,0.077671
97,0.0,1.0,0.01,18.836739,0.140923,0.300192,0.041097,0.086127,0.076817
98,0.0,1.0,0.01,18.998763,0.146176,0.303223,0.041776,0.087531,0.076209


In [6]:
add_arithm_mean(replicates)
add_geom_mean(replicates)
add_glv_rhs(replicates)
add_log_glv_rhs(replicates)
replicates

,dataset,perturbation_scale,measurement_noise,time,sp1,sp2,sp3,sp4,sp5,geom_mean_sp1,...,arith_mean_sp1,arith_mean_sp2,arith_mean_sp3,arith_mean_sp4,arith_mean_sp5,dsp1/dt,dsp2/dt,dsp3/dt,dsp4/dt,dsp5/dt
0,0.0,1.0,0.01,0.180160,0.199071,0.426655,0.028356,0.052023,0.111200,0.201753,...,0.201771,0.419448,0.028795,0.053746,0.111091,0.101459,-0.270819,0.016483,0.064734,-0.004082
1,0.0,1.0,0.01,0.233384,0.204471,0.412241,0.029233,0.055469,0.110983,0.201177,...,0.201203,0.381908,0.029331,0.057076,0.106743,-0.018023,-0.167287,0.000538,0.008865,-0.023384
2,0.0,1.0,0.01,0.596027,0.197935,0.351575,0.029428,0.058684,0.102503,0.198063,...,0.198063,0.349586,0.029921,0.058232,0.102245,0.002930,-0.045532,0.011265,-0.010325,-0.005904
3,0.0,1.0,0.01,0.683426,0.198191,0.347596,0.030413,0.057781,0.101987,0.196415,...,0.196423,0.340056,0.030870,0.059203,0.100399,-0.015824,-0.067474,0.004087,0.012721,-0.014211
4,0.0,1.0,0.01,0.906915,0.194655,0.332516,0.031326,0.060624,0.098811,0.193759,...,0.193761,0.328193,0.031568,0.060949,0.099043,-0.006433,-0.031106,0.001739,0.002339,0.001675
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,2.0,1.0,0.01,18.697444,0.153701,0.293866,0.040299,0.091128,0.068952,0.154281,...,0.154282,0.293803,0.040444,0.090421,0.066956,0.003826,-0.000418,0.000950,-0.004659,-0.013156
296,2.0,1.0,0.01,19.000866,0.154862,0.293740,0.040588,0.089714,0.064960,0.154123,...,0.154125,0.296115,0.040730,0.090409,0.064830,-0.026459,0.085205,0.005089,0.024925,-0.004657
297,2.0,1.0,0.01,19.056622,0.153387,0.298490,0.040871,0.091104,0.064700,0.152196,...,0.152200,0.296296,0.040910,0.090525,0.065076,-0.013988,-0.025862,0.000459,-0.006818,0.004420
298,2.0,1.0,0.01,19.226345,0.151013,0.294101,0.040949,0.089947,0.065451,0.151571,...,0.151572,0.290736,0.041157,0.089671,0.065317,0.012669,-0.076315,0.004701,-0.006250,-0.003023
